In [1]:
import os,cv2
import numpy as np
import json

with open('Data/train/train.json') as f:
  data = json.load(f)[200:400]

In [2]:
def get_iou(bb1, bb2):
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    return iou

def P_transform(bbx):
    x,y,w,h = bbx
    # x,y,w,h
    return(np.array([x+w/2,y+h/2,float(w),float(h)]))

def G_transform(bbx):
    x1,x2,y1,y2 = bbx['x1'],bbx['x2'],bbx['y1'],bbx['y2']
    # x,y,w,h
    return(np.array([(x2+x1)/2,(y2+y1)/2,(x2-x1),(y2-y1)]))

In [3]:
threshold = 0.85
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

G,P,regions = [],[],[]

for i in range(len(data)):
    subject = data[i]
    path = 'Data/train'+subject['image']['pathname']
    image = cv2.imread(path)
    rects = subject['objects']
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssrects = ss.process()
    ssrects = ssrects[np.random.RandomState(seed=i).permutation(len(ssrects))[:2000]]
    imout = image.copy()
    
    for j in range(len(rects)):
        box = rects[j]['bbx']
        iou = np.array([get_iou(box,{"x1":x,"x2":x+w,"y1":y,"y2":y+h}) for x,y,w,h in ssrects])
        ind = np.argmax(iou)
        iou = np.max(iou)
        
        if iou<threshold:
            continue
            
        p = ssrects[ind]
        x,y,w,h = p
        region = cv2.resize(imout[y:y+h,x:x+w], (224,224), interpolation = cv2.INTER_AREA)
        
        G.append(G_transform(box))
        P.append(P_transform(p))
        regions.append(region.astype(float)/225)

In [4]:
print(len(G))

2915


#### Extract features

In [5]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
 transforms.ToTensor(),
 transforms.Normalize(
 mean,std)])

def data_loader(X,y):
    d_ts = [transform(x) for x in X]
    d_ls = []
    for i in range(len(d_ts)):
        d_ls.append([d_ts[i],int(y[i])])
    loader = DataLoader(dataset = d_ls,batch_size=1,shuffle=False)
    return(loader)

torch.manual_seed(1)

data = data_loader(regions,range(regions.shape[0]))

del regions

In [6]:
import torchvision.models as models
import torch.nn as nn

alexnet = models.alexnet()
alexnet.classifier = nn.Sequential(*list(alexnet.classifier[:6]))
alexnet.load_state_dict(torch.load('Model/alexnet.pth'))

<All keys matched successfully>

In [7]:
phi = []

alexnet.eval()
for feature,label in data:
  phi.append(np.array(alexnet(feature.float()).detach().numpy()).reshape([4096]))

phi = np.array(phi)

del data

#### Get  $\ \ t_*\ \ $  variables

In [8]:
import pandas as pd

G,P = pd.DataFrame(G,columns=['x','y','w','h']),pd.DataFrame(P,columns=['x','y','w','h'])

In [9]:
t_x = (G['x']-P['x'])/P['w']
t_y = (G['y']-P['y'])/P['h']
t_w = np.log(G['w']/P['w'])
t_h = np.log(G['h']/P['h'])

T = pd.concat([t_x,t_y,t_w,t_h],axis=1)
T.columns = ['t_x','t_y','t_w','t_h']

del G,P

#### Save data

In [10]:
np.save("Data/BBR/phi",phi)
T.to_csv("Data/BBR/T.csv")